# Preprocessing the Yahoo! Dataset

# Functions for turning tracks to artists

In [1]:
# Paths to relevant files

albumFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/albumData1.txt'
trackFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/trackData1.txt'
artistFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/artistData1.txt'
genreFile = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/genreData1.txt'
##mode = 'r' #r for read, w for write only, a for append, r+ for r+w 
#f1 = open(trackFile,mode) 
#f2 = open(artistFile,mode) 

# Transform a trackID to artistID
# PRE: itemID is confirmed to be of type in targetFile
# If no match found, returns None
def trackToArtist(itemID,trackF):
    trackID = int(itemID)
    file = open(trackF,'r')
    for line in file:
        splitLine = line.split('|')
        if len(splitLine) < 3: 
            print('Error: Too little info about track')
            break
        lineID = int(splitLine[0].rstrip())
        artistID = splitLine[2].rstrip()
    
        if trackID == lineID:
            #Have found a match
            #Check if artist data exists
            if artistID == 'None':
                return None
            else:
                return int(artistID)
        elif trackID < lineID:
            #Because the files are sorted we can terminate when we pass the sought for value
            #print('test')
            #We should NOT end up here
            print('Error, item is not a track')    
            return None    
    # Should never reach these lines
    print('Error: Nothing descisive found')
    return None
        

print('Test1: %s' % (trackToArtist('0',trackFile) == 587636))
print('Test2: %s' % (trackToArtist(7,trackFile)  == None))
print('Test3: %s' % (trackToArtist(80,trackFile)  == None))
print('Test4: %s' % (trackToArtist('624960',trackFile) == 380811))

# Check if item itemID is of the type specified by typeFile
# Assumption: Because the files are sorted we can terminate when we pass the sought for value
def isOfType(inputID,fileName):
    itemID = int(inputID)
    #print(itemID)
    file = open(fileName,'r')
    
    for line in file:
        splitLine = line.split('|')
        lineID = int(splitLine[0].rstrip())
        #print(lineID)
        if itemID == lineID:
            return True
        elif itemID < lineID:
            #Because the files are sorted we can terminate when we pass the sought for value
            #print('test')
            return False
    
    return False


print('Test5: %s' % (isOfType('5',trackFile) == True))
print('Test6: %s' % (isOfType('10',trackFile) == False))
print('Test7: %s' % (isOfType('10',artistFile) == True))
print('Test8: %s' % (isOfType('200',artistFile) == False))


Test1: True
Test2: True
Error, item is not a track
Test3: True
Test4: True
Test5: True
Test6: True
Test7: True
Test8: True


# Functions for: Create User Data and turn to Ngrams

In [2]:
# Tracked: trackId, rating, date, time
# WARNING: PRE: User data is one item, can not handle inf large rating-item lists
def createUserDataItem(inputFile,nrDataTracked,nrRatings):
    userData = [[0 for x in range(0,nrRatings)] for y in range(0,nrDataTracked)]
    # Called: userData[3][39] 3: item index, 39: is raingIndex
    # Uses zero-indexing
    for x in range(0,int(nrRatings)):
        line = inputFile.readline()
        tabSplit = line.split('\t')
        if len(tabSplit) == 4: #DEFFENSIVE should not be anything else
            #We have a rating-line
            # Save to userData:
            #trackID  
            userData[0][x] = int(tabSplit[0])        
            #rating
            userData[1][x] = int(tabSplit[1])
            #date 
            userData[2][x] = int(tabSplit[2])
            #time 
            userData[3][x] = tabSplit[3].rstrip()
        else:
            print('ERROR: rating-item line expected in user data creation')
    return userData

# Transforms single-user data to NGRAM and writes it to outputFile
# Uses limitations of what to count to ngram 
def userDataToNgram(ratingDiffDown,ratingDiffUpp,dateLimitDown,dateLimitUpp,outputFile):
    counter = 0 # REMOVE later, only for testing
    ngrams = ''
    
    # Take 'next' song and compare with all other to find corrolations
    #For each item, compare with all other
    for index in range(0,nrRatings):
        #PRE: data is sorted on date, cronologically 
        #Fetch item data
        trackID = userData[0][index]
        rating = userData[1][index]
        date = userData[2][index]
        time = userData[3][index]

        #Compare it on conditions to other ratings
        for i in range(0,nrRatings):
            #Avoid comparing with self
            if i != index:
                #Check date limits
                if date-dateLimitDown <= userData[2][i] and userData[2][i] <= date+dateLimitUpp:
                    #Check rating limits
                    if rating-ratingDiffDown <= userData[1][i] and userData[1][i] <= rating+ratingDiffUpp:
                        counter = counter + 1
                        #All limits are passed, save item correlations
                        outString = '1,%s,%s\n' % (trackID, userData[0][i])
                        #ngrams += outString
                        outputFile.write(outString)
                        
    #outputFile.write(ngrams)
    return counter

In [23]:
# Import libraries needed
import os
import time

# Contains variables to set and experiment with
dateLimitDown = 0 #Nr dates before current rating to include
dateLimitUpp = 0 #Nr dates after current rating to include
ratingDiffDown = 0 #0 to 100, 0 means equal rating only 100 means all below included 
ratingDiffUpp = 0 #0 to 100, 0 means equal rating only 100 means all above included
# ---------------------

maxNrUsers   = 100 #Meta: nUsers= 1000990


# READ or RE-READ the file
#filename = 'dummy-2-users.txt'
#filename = 'Webscope_C15/ydata-ymusic-kddcup-2011-track1/trainIdx1.txt'
filename = 'data/temp/artistDataset_10000_users.txt'


# ---------------------
mode = 'r' #r for read, w for write only, a for append, r+ for r+w 
f = open(filename,mode) 


# Open WRITE file 
outputFile = 'output_100.txt'
modeOut = 'a' #r for read, w for write only, a for append, r+ for r+w 
#os.remove(outputFile) #TODO: Is this really ok structure to have? 
fOut = open(outputFile,modeOut) 


#FIXED VALUES:
nrDataTracked = 4 #File structure info for cration of user data object DO NOT TOUCH


# Run script:

In [24]:
start_time = time.time()

for user in range(0,maxNrUsers): 
    # READ next line, should be a user
    line = f.readline()
    
    #Deffensive: Check for empty file  
    if line == '':
        print('End of file reached')
        #print(out)
        # Close used files
        fOut.close()
        f.close()
        break
    else:
        #Know we have a non-empty line        
        out = 'initial value'
        lineSplit = line.split('|')
        tabSplit = line.split('\t')

        # Checking to make sure of userID line
        if len(lineSplit) == 2 and len(tabSplit) == 1: 
            #Got a user, extracting meta-data
            userID = lineSplit[0]
            nrRatings = int(lineSplit[1].rstrip())
            #out = userID

            #Create user data from input file ratings 
            userData = createUserDataItem(f,nrDataTracked,nrRatings)
            
            #print(userData)
            
            #Transform user data to ngrams and save to file
            out = userDataToNgram(ratingDiffDown,ratingDiffUpp,dateLimitDown,dateLimitUpp,fOut)
            
        else : 
            #Deffensive to make sure correct use of file structure
            print('ERROR: in file read process, user info line expected')
            break
    #print(out)
    if user % 100 == 0:
        print(user)
        
# Close used files
fOut.close()
f.close()
print('Done')
end_time=time.time()-start_time
print(end_time)

# 1000 users => 250 MB
# 1000 users (nr2) => 250 MB in 364 sec = 6 min :: NOTE: AFTER CASHING 
# 1000 users = 361 sec

# 10000 users => 2.64 GB in 2690 sec = 44 min :: NOTE: before cashing
# 10000 users =>            2676 sec             NOTE: after cashing


# Prediction, linearly scaling: 
#   100 000 users =>  26 GB in  7.5 hours
# 1 000 000 users => 260 GB in 75   hours = 3.1 days

# That is BEFORE: data storage aggregation and "cashing" of user data before write to disk


0
Done
14.970263004302979


In [ ]:
1+2